In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np

In [0]:
VOCAB_SIZE = 88584
MAXLEN = 250
BATCH_SIZE = 64

In [5]:
(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

17465344/17464789 [==============================] - 0s 0us/step


In [0]:
train_data = sequence.pad_sequences(train_data,MAXLEN)
test_data =  sequence.pad_sequences(test_data,MAXLEN)

In [0]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Embedding(VOCAB_SIZE,32),
     tf.keras.layers.LSTM(32),
     tf.keras.layers.Dense(1,activation="sigmoid")
    ]
)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)

In [12]:
history = model.fit(train_data,train_labels,epochs=10,validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 63s 3ms/sample - loss: 0.4297 - accuracy: 0.7993 - val_loss: 0.2854 - val_accuracy: 0.8834
Epoch 2/10
20000/20000 [==============================] - 62s 3ms/sample - loss: 0.2433 - accuracy: 0.9072 - val_loss: 0.3140 - val_accuracy: 0.8776
Epoch 3/10
20000/20000 [==============================] - 61s 3ms/sample - loss: 0.1837 - accuracy: 0.9326 - val_loss: 0.2697 - val_accuracy: 0.8918
Epoch 4/10
20000/20000 [==============================] - 61s 3ms/sample - loss: 0.1519 - accuracy: 0.9449 - val_loss: 0.2743 - val_accuracy: 0.8910
Epoch 5/10
20000/20000 [==============================] - 61s 3ms/sample - loss: 0.1280 - accuracy: 0.9568 - val_loss: 0.4235 - val_accuracy: 0.8304
Epoch 6/10
20000/20000 [==============================] - 61s 3ms/sample - loss: 0.1127 - accuracy: 0.9620 - val_loss: 0.2945 - val_accuracy: 0.8874
Epoch 7/10
20000/20000 [==============================] -

In [14]:
results = model.evaluate(test_data,test_labels)
print(results)

25000/25000 [==============================] - 18s 701us/sample - loss: 0.4991 - accuracy: 0.8519
[0.49907939080238345, 0.85192]


### Predictions

In [21]:
#Encoding function

word_index = imdb.get_word_index()

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens],MAXLEN)[0]

text ="that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [22]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [23]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.9542549]
[0.2779409]
